In [1]:
from pymongo import MongoClient

In [2]:
host="rhea.isegi.unl.pt"
port="28005"
user="GROUP_5"
password="MzA0MTk3MTEwNTM2Mzg3NTY4MzMxODIzODUzNzQzNzcwOTAwNjkz"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [3]:
db = client.contracts
print(f"Database info: {db}\n")

Database info: Database(MongoClient(host=['rhea.isegi.unl.pt:28005'], document_class=dict, tz_aware=False, connect=True), 'contracts')



In [4]:
result1 = db.eu.find().count()
result1

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


5563415

In [5]:
#group cpv div (in the cpv collection) and output a new collection "cpv_grouped" with only the first two digits (cpv division) and the cpv description

query1={"$project":{"cpv_division":1,"cpv_division_description":1}}

query2={"$group": {"_id":{"cpv_division":"$cpv_division"},"cpv_division_description":{"$addToSet":"$cpv_division_description"}}}

query3={"$out":"cpv_grouped"}

pipeline=[query1, query2,query3]
grouped_cpv=list(db.cpv.aggregate(pipeline))

In [6]:
db.cpv_grouped.find_one()

{'_id': {'cpv_division': '44'},
 'cpv_division_description': ['Construction structures and materials; auxiliary products to construction (except electric apparatus)']}

In [7]:
#get the first two digits of the cpv in eu collection (like in assignment2)
switch={"$switch":{
    "branches":[
        {"case":{"$lt":["$CPV",1000000]},"then":"00"},
        {"case":{"$lt":["$CPV",10000000]},"then":{"$substr":["$CPV", 0, 1]}}
    ],
    "default": {"$substr":["$CPV", 0, 2]}
}}


In [8]:
#try to merge (embedded model) cpv_grouped with eu #CURRENTLY NOT WORKING RESPECTIVELY TAKES TOO MUCH TIME 
query1= {"$lookup":{
           "from": "cpv_grouped",
           "localField": str(switch),
           "foreignField": "_id",
           "as":"lookup_eu_cpv"
        }
    }
    
query2={"$merge" : {
            "into" : "eu_cpv",
            "whenMatched": "keepExisting"
        } 
    }

pipeline=[query1, query2]
eu_cpv=list(db.eu.aggregate(pipeline))

KeyboardInterrupt: 

### 1.1 Each CPV’s division contracts average spending (‘VALUE_EURO’)


In [8]:

query1={"$project":{"VALUE_EURO":1,"CPV":switch}}

query2={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query3={"$group": {"_id":{},
                "avgValueEuro":{"$avg":"$avgValueEuro"}
                 }
       }

pipeline=[query1, query2,query3]
result_1_1=list(db.eu.aggregate(pipeline))

In [ ]:
result_1_1

## 1.2 Each CPV’s division contract count

In [53]:
query1={"$project":{"CPV":switch}}

query2={"$group": {"_id":{"CPV":"$CPV"},
                "count":{"$sum":1}}}

query3={"$group": {"_id":{},
                "avgCount":{"$avg":"$count"}
                 }
       }


pipeline=[query1, query2,query3]
result_1_2=list(db.eu.aggregate(pipeline))

In [54]:
result_1_2

[{'_id': {}, 'avgCount': 120943.76086956522}]

In [ ]:
### 1.3 Each CPV’s division contracts average number of offers (‘NUMBER_OFFERS’)

In [56]:
query1={"$project":{"CPV":switch,"NUMBER_OFFERS":1}}

query2={"$group": {"_id":{"CPV":"$CPV"},
                "avgOffers":{"$avg":"$NUMBER_OFFERS"}}}

query3={"$group": {"_id":{},
                "avgOffers":{"$avg":"$avgOffers"}
                 }
       }

pipeline=[query1, query2, query3]
result_1_3=list(db.eu.aggregate(pipeline))

In [57]:
result_1_3

[{'_id': {}, 'avgOffers': 7.572088391451659}]

In [58]:
query1={"$match":{"B_EU_FUNDS":{"$eq":"Y"}}}

query2={"$project":{"VALUE_EURO":1,"CPV":switch}}

query3={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query4={"$group": {"_id":{},
                "avgValueEuro":{"$avg":"$avgValueEuro"}
                 }
       }

pipeline=[query1,query2, query3,query4]
result_1_4=list(db.eu.aggregate(pipeline))

In [59]:
result_1_4

[{'_id': {}, 'avgValueEuro': 1.2022182214644884e+22}]

### 1.5 Each CPV’s division contracts average spending (‘VALUE_EURO’) without european funds (‘B_EU_FUNDS’)

In [62]:
query1={"$match":{"$or":[{"B_EU_FUNDS":"N"},{"B_EU_FUNDS":None}]}}

query2={"$project":{"VALUE_EURO":1,"CPV":switch}}

query3={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query4={"$group": {"_id":{},
                "avgValueEuro":{"$avg":"$avgValueEuro"}}}

pipeline=[query1,query2,query3,query4]
result_1_5=list(db.eu.aggregate(pipeline))

In [63]:
result_1_5

[{'_id': {}, 'avgValueEuro': 1.2818269566081569e+19}]

### 2 The count of contracts for each CPV Division

In [8]:
#it doesn't display the cpv_division_description; referrenced model (lookup) takes long 
query0={"$lookup":{
           "from": "cpv_grouped",
           "localField": str(switch),
           "foreignField": "cpv_division",
           "as": "CPV_name"
        }}

query1={"$project":{"cpv_division_description":1}}

query2={"$group": {"_id":{"cpv_division_description":"$cpv_division_description"},
                "count":{"$sum":1}}}

pipeline=[query0,query1,query2]
result_2=list(db.eu.aggregate(pipeline))

In [66]:
result_2

[{'_id': {'CPV': '44'}, 'count': 81992},
 {'_id': {'CPV': '50'}, 'count': 183605},
 {'_id': {'CPV': '30'}, 'count': 128660},
 {'_id': {'CPV': '75'}, 'count': 14389},
 {'_id': {'CPV': '66'}, 'count': 116734},
 {'_id': {'CPV': '65'}, 'count': 13465},
 {'_id': {'CPV': '34'}, 'count': 175304},
 {'_id': {'CPV': '73'}, 'count': 18992},
 {'_id': {'CPV': '31'}, 'count': 52955},
 {'_id': {'CPV': '70'}, 'count': 11924},
 {'_id': {'CPV': '33'}, 'count': 1775459},
 {'_id': {'CPV': '15'}, 'count': 252972},
 {'_id': {'CPV': '79'}, 'count': 204709},
 {'_id': {'CPV': '90'}, 'count': 235427},
 {'_id': {'CPV': '32'}, 'count': 38861},
 {'_id': {'CPV': '76'}, 'count': 3155},
 {'_id': {'CPV': '35'}, 'count': 26641},
 {'_id': {'CPV': '80'}, 'count': 131271},
 {'_id': {'CPV': '72'}, 'count': 99613},
 {'_id': {'CPV': '42'}, 'count': 48194},
 {'_id': {'CPV': '98'}, 'count': 25391},
 {'_id': {'CPV': '92'}, 'count': 20517},
 {'_id': {'CPV': '16'}, 'count': 8035},
 {'_id': {'CPV': '24'}, 'count': 41502},
 {'_id':

### 3 Per CPV Division get the average spending (‘VALUE_EURO’) and return the highest 5 cpvs

In [67]:
#it doesn't display the cpv_division_description;
query1={"$project":{"VALUE_EURO":1,"CPV":switch}}

query2={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query3={"$sort": {"avgValueEuro":-1}}

query4={"$limit":5}

pipeline=[query1, query2,query3,query4]
result_3=list(db.eu.aggregate(pipeline))

In [68]:
result_3

[{'_id': {'CPV': '90'}, 'avgValueEuro': 1.1423173092228093e+22},
 {'_id': {'CPV': '30'}, 'avgValueEuro': 4.4276521636461806e+20},
 {'_id': {'CPV': '75'}, 'avgValueEuro': 1.0979761462251373e+19},
 {'_id': {'CPV': '79'}, 'avgValueEuro': 3.572801959007498e+17},
 {'_id': {'CPV': '80'}, 'avgValueEuro': 233961466170367.25}]

### 4 Per CPV Division get the average spending (‘VALUE_EURO’) and return the lowest 5 cpvs

In [69]:
#it doesn't display the cpv_division_description;
query1={"$project":{"VALUE_EURO":1,"CPV":switch}}

query2={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query3={"$sort": {"avgValueEuro":1}}

query4={"$limit":5}

pipeline=[query1, query2,query3,query4]
result_4=list(db.eu.aggregate(pipeline))

In [70]:
result_4

[{'_id': {'CPV': '3'}, 'avgValueEuro': 1261617.4059277952},
 {'_id': {'CPV': '16'}, 'avgValueEuro': 1666574.1648330106},
 {'_id': {'CPV': '37'}, 'avgValueEuro': 1672027.3621667938},
 {'_id': {'CPV': '92'}, 'avgValueEuro': 1961491.3667497546},
 {'_id': {'CPV': '19'}, 'avgValueEuro': 1971553.785299904}]

### 5 Per CPV Division get the average spending (‘VALUE_EURO’) and return the highest 5 cpvs for contracts which recieved european funds (‘B_EU_FUNDS’)

In [71]:
#it doesn't display the cpv_division_description;
query1={"$match":{"B_EU_FUNDS":{"$eq":"Y"}}}

query2={"$project":{"VALUE_EURO":1,"CPV":switch}}

query3={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query4={"$sort": {"avgValueEuro":-1}}

query5={"$limit":5}

pipeline=[query1,query2, query3,query4,query5]
result_5=list(db.eu.aggregate(pipeline))

In [72]:
result_5

[{'_id': {'CPV': '55'}, 'avgValueEuro': 213072.63850953206},
 {'_id': {'CPV': '18'}, 'avgValueEuro': 236327.45987437185},
 {'_id': {'CPV': '19'}, 'avgValueEuro': 290370.24028985505},
 {'_id': {'CPV': '16'}, 'avgValueEuro': 352969.07318627456},
 {'_id': {'CPV': '92'}, 'avgValueEuro': 479732.1283990442}]

### 6 Per CPV Division and get the average (‘VALUE_EURO’) return the highest 5 cpvs for contracts which did not recieve european funds (‘B_EU_FUNDS’)

In [73]:

#it doesn't display the cpv_division_description;
query1={"$match":{"$or":[{"B_EU_FUNDS":"N"},{"B_EU_FUNDS":None}]}}

query2={"$project":{"VALUE_EURO":1,"CPV":switch}}

query3={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query4={"$sort": {"avgValueEuro":-1}}

query5={"$limit":5}

pipeline=[query1,query2,query3,query4,query5]
result_6=list(db.eu.aggregate(pipeline))

In [74]:
result_6

[{'_id': {'CPV': '30'}, 'avgValueEuro': 5.773672055427442e+20},
 {'_id': {'CPV': '75'}, 'avgValueEuro': 1.1841854326803847e+19},
 {'_id': {'CPV': '79'}, 'avgValueEuro': 4.3133068536093466e+17},
 {'_id': {'CPV': '73'}, 'avgValueEuro': 6354932275958.761},
 {'_id': {'CPV': '85'}, 'avgValueEuro': 1778557507080.2217}]

### 7 The highest CPV Division on average spending (‘VALUE_EURO’) per country (‘ISO_COUNTRY_CODE’)

In [20]:
#contains duplicates within country code 
query1={"$project":{"VALUE_EURO":1,"CPV":switch,"ISO_COUNTRY_CODE":1}}

query2={"$group": {"_id":{"CPV":"$CPV","ISO_COUNTRY_CODE":"$ISO_COUNTRY_CODE"},"avgValueEuro":{"$avg":"$VALUE_EURO"}}}

query3={"$group": {"_id":"$_id","maxAvgValueEuro":{"$max":"$avgValueEuro"}}}

pipeline=[query1,query2,query3]
result_7=list(db.eu.aggregate(pipeline))

In [21]:
result_7

03461},
 {'_id': {'CPV': '34', 'ISO_COUNTRY_CODE': 'NL'},
  'maxAvgValueEuro': 3163748.115686813},
 {'_id': {'CPV': '39', 'ISO_COUNTRY_CODE': 'AT'},
  'maxAvgValueEuro': 1623986.0301470587},
 {'_id': {'CPV': '33', 'ISO_COUNTRY_CODE': 'SI'},
  'maxAvgValueEuro': 1572955.0385864126},
 {'_id': {'CPV': '72', 'ISO_COUNTRY_CODE': 'CH'},
  'maxAvgValueEuro': 11329385.923194446},
 {'_id': {'CPV': '15', 'ISO_COUNTRY_CODE': 'FR'},
  'maxAvgValueEuro': 5637779.305134567},
 {'_id': {'CPV': '55', 'ISO_COUNTRY_CODE': 'CY'},
  'maxAvgValueEuro': 1039556.7225},
 {'_id': {'CPV': '37', 'ISO_COUNTRY_CODE': 'SE'},
  'maxAvgValueEuro': 779300.4134000001},
 {'_id': {'CPV': '75', 'ISO_COUNTRY_CODE': 'CH'},
  'maxAvgValueEuro': 1511557.805652174},
 {'_id': {'CPV': '30', 'ISO_COUNTRY_CODE': 'NL'},
  'maxAvgValueEuro': 5613324.000976492},
 {'_id': {'CPV': '63', 'ISO_COUNTRY_CODE': 'UK'},
  'maxAvgValueEuro': 66001437.06311688},
 {'_id': {'CPV': '79', 'ISO_COUNTRY_CODE': 'CY'},
  'maxAvgValueEuro': 1615910.58632

### 8 Returns bucketed data with the contract counts of a particular cpv in a given range of values (bucket) according to spending (‘VALUE_EURO’)

In [17]:
query1={
    "$bucket": {
      "groupBy": "$VALUE_EURO",                        #// Field to group by
      "boundaries": [0,1000000, 2000000, 3000000, 4000000, 5000000,6000000,7000000,8000000,9000000,10000000 ], #// Boundaries
      "default": ">10Mil",                             #// Bucket id for documents which do not fall into a bucket
      "output": {                                     #// Output for each bucket
        "count": { "$sum": 1 }}}}

pipeline=[query1]
result_8=list(db.eue.aggregate(pipeline))

In [107]:
result_8

[{'_id': 0, 'count': 1968422},
 {'_id': 1000000, 'count': 374742},
 {'_id': 2000000, 'count': 158206},
 {'_id': 3000000, 'count': 98708},
 {'_id': 4000000, 'count': 66676},
 {'_id': 5000000, 'count': 56835},
 {'_id': 6000000, 'count': 38594},
 {'_id': 7000000, 'count': 29199},
 {'_id': 8000000, 'count': 26977},
 {'_id': 9000000, 'count': 20178},
 {'_id': '>10Mil', 'count': 2724876}]

### 9 The average time and value difference for each CPV, return the highest 5 cpvs

In [126]:
#awardDate and dispatchDate are datetime but it doesn't compute the timedifference 

query1={"$project":{"CPV":switch, "awardDate":{"$toDate":"$DT_AWARD"}, "dispatchDate":{"$toDate": "$DT_DISPATCH"}, "time":{"$subtract":["$dispatchDate","$awardDate"]},"value":{"$subtract":["$AWARD_VALUE_EURO","$VALUE_EURO"]}}}

query2={"$limit":5}

query2={"$group": {"_id":{"CPV":"$CPV"},
                "avgValueDiff":{"$avg":"$value"},"avgTimeDiff":{"$avg":"$time"}}}

query3={"$sort":{"time":-1}}

pipeline=[query1, query2, query3]

result_9=list(db.eu.aggregate(pipeline))





In [127]:
result_9

[{'_id': ObjectId('5e58fe88e3912091230e07a2'),
  'CPV': '79',
  'awardDate': datetime.datetime(2014, 11, 20, 0, 0),
  'dispatchDate': datetime.datetime(2014, 12, 23, 0, 0),
  'time': None,
  'value': 0},
 {'_id': ObjectId('5e58fe88e3912091230e07a3'),
  'CPV': '72',
  'awardDate': None,
  'dispatchDate': datetime.datetime(2017, 12, 22, 0, 0),
  'time': None,
  'value': None},
 {'_id': ObjectId('5e58fe88e3912091230e07a4'),
  'CPV': '30',
  'awardDate': datetime.datetime(2009, 10, 8, 0, 0),
  'dispatchDate': datetime.datetime(2009, 12, 23, 0, 0),
  'time': None,
  'value': -125000},
 {'_id': ObjectId('5e58fe88e3912091230e07a5'),
  'CPV': '73',
  'awardDate': datetime.datetime(2009, 11, 27, 0, 0),
  'dispatchDate': datetime.datetime(2009, 12, 21, 0, 0),
  'time': None,
  'value': 0},
 {'_id': ObjectId('5e58fe88e3912091230e07a6'),
  'CPV': '79',
  'awardDate': datetime.datetime(2009, 11, 26, 0, 0),
  'dispatchDate': datetime.datetime(2009, 12, 21, 0, 0),
  'time': None,
  'value': -100000}]